# CSE 144 Group 3
## Variational Autoencoder

In [320]:
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
%config InlineBackend.figure_format="retina"
import numpy as np
import random
import torch
from torch import nn, optim
import math
from IPython import display
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset
import torch.nn.functional as F
from sklearn.preprocessing import MinMaxScaler
import pdb

In [321]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [322]:
torch.manual_seed(24)

In [323]:
print(len(data))
data.head()

170653


,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
0,0.0594,1921,0.982,"['Sergei Rachmaninoff', 'James Levine', 'Berli...",0.279,831667,0.211,0,4BJqT0PrAfrxzMOxytFOIz,0.878000,10,0.665,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921,0.0366,80.954
1,0.9630,1921,0.732,['Dennis Day'],0.819,180533,0.341,0,7xPhfUan2yNtyFG0cUWkt8,0.000000,7,0.160,-12.441,1,Clancy Lowered the Boom,5,1921,0.4150,60.936
2,0.0394,1921,0.961,['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...,0.328,500062,0.166,0,1o6I8BglA6ylDMrIELygv1,0.913000,3,0.101,-14.850,1,Gati Bali,5,1921,0.0339,110.339
3,0.1650,1921,0.967,['Frank Parker'],0.275,210000,0.309,0,3ftBPsC5vPBKxYSee08FDH,0.000028,5,0.381,-9.316,1,Danny Boy,3,1921,0.0354,100.109
4,0.2530,1921,0.957,['Phil Regan'],0.418,166693,0.193,0,4d6HGyGT8e121BsdKmw9v6,0.000002,3,0.229,-10.096,1,When Irish Eyes Are Smiling,2,1921,0.0380,101.665


# Data Preprocessing


In [324]:
data["artists"] = data["artists"].str.replace("[", "")
data["artists"] = data["artists"].str.replace("]", "")
data["artists"] = data["artists"].str.replace("'", "")
data["artists"] = data["artists"].map(lambda row: row.split(', '))

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170653 entries, 0 to 170652
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   valence           170653 non-null  float64
 1   year              170653 non-null  int64  
 2   acousticness      170653 non-null  float64
 3   artists           170653 non-null  object 
 4   danceability      170653 non-null  float64
 5   duration_ms       170653 non-null  int64  
 6   energy            170653 non-null  float64
 7   explicit          170653 non-null  int64  
 8   id                170653 non-null  object 
 9   instrumentalness  170653 non-null  float64
 10  key               170653 non-null  int64  
 11  liveness          170653 non-null  float64
 12  loudness          170653 non-null  float64
 13  mode              170653 non-null  int64  
 14  name              170653 non-null  object 
 15  popularity        170653 non-null  int64  
 16  release_date      17

In [325]:
eleven_features = ['danceability', 'duration_ms', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness',
                   'tempo', 'mode', 'valence', 'acousticness']
data = data[data.columns.intersection(eleven_features)]
data.head()

,valence,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,mode,speechiness,tempo
0,0.0594,0.982,0.279,831667,0.211,0.878000,0.665,-20.096,1,0.0366,80.954
1,0.9630,0.732,0.819,180533,0.341,0.000000,0.160,-12.441,1,0.4150,60.936
2,0.0394,0.961,0.328,500062,0.166,0.913000,0.101,-14.850,1,0.0339,110.339
3,0.1650,0.967,0.275,210000,0.309,0.000028,0.381,-9.316,1,0.0354,100.109
4,0.2530,0.957,0.418,166693,0.193,0.000002,0.229,-10.096,1,0.0380,101.665


In [326]:
x = np.array(data['tempo'])
y = np.array(data['loudness'])
scaler = MinMaxScaler()
data['tempo'] = scaler.fit_transform(x[:, np.newaxis])
data['loudness'] = scaler.fit_transform(y[:, np.newaxis])
data.head()

<ipython-input-326-8c77edf1f0ec>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['tempo'] = scaler.fit_transform(x[:, np.newaxis])
<ipython-input-326-8c77edf1f0ec>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['loudness'] = scaler.fit_transform(y[:, np.newaxis])


,valence,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,mode,speechiness,tempo
0,0.0594,0.982,0.279,831667,0.211,0.878000,0.665,0.624916,1,0.0366,0.332450
1,0.9630,0.732,0.819,180533,0.341,0.000000,0.160,0.744797,1,0.4150,0.250243
2,0.0394,0.961,0.328,500062,0.166,0.913000,0.101,0.707071,1,0.0339,0.453125
3,0.1650,0.967,0.275,210000,0.309,0.000028,0.381,0.793736,1,0.0354,0.411113
4,0.2530,0.957,0.418,166693,0.193,0.000002,0.229,0.781521,1,0.0380,0.417503


In [327]:
size_of_train = int(0.7 * len(data))
print(size_of_train)
size_of_test= len(data) - size_of_train

tensor = torch.from_numpy(np.array(data))

train_dataset, test_dataset = torch.utils.data.random_split(tensor, (size_of_train, size_of_test))

# size_of_train = int(0.9 * len(train_dataset))
# size_of_val = len(train_dataset) - size_of_train

# val_dataset = TensorDataset(torch.from_numpy(val_dataset.numpy()))
# train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, (size_of_train, size_of_val))

train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=1000, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset,
                                              batch_size=1000, shuffle=False)
# val_loader = torch.utils.data.DataLoader(val_dataset,
#                                              batch_size=100, shuffle=False)


119457


In [328]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [329]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
                                        # ENCODER
        self.fc1 = nn.Linear(11, 128)   # 11 input song features to 128-D hidden
                                        # vector
        self.fc21 = nn.Linear(128, 150) # hidden vector -> std vector
        self.fc22 = nn.Linear(128, 150) # hidden vector -> mean vector

                                        # DECODER
        self.fc3 = nn.Linear(150, 128)  # latent vector -> 128-D hidden
                                        # vector

        self.fc4 = nn.Linear(128, 11)   # 128-D hidden vector -> 11 output

    def encode(self, x):
        x = F.tanh(self.fc1(x))
        # x = torch.flatten(x, start_dim=1)
        return [self.fc21(x), self.fc22(x)]

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        return torch.sigmoid(self.fc4(F.tanh(self.fc3(z))))

    def forward(self, input):
        x = input
        mu, logvar = self.encode(x)[0], self.encode(x)[1]
        z = self.reparameterize(mu, logvar)
        return [self.decode(z), input, mu, logvar]


In [330]:
def train_model(training_loader, learning_rate=1e-3, epochs=1):
  model = VAE()
  model.train()

  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  criterion = nn.MSELoss()
  model = model.to(device).float()

  recon_losses = []
  kld_losses = []
  losses = []

  for epoch in range(epochs):
      for batch_idx, input in enumerate(train_loader):
          # Get a batch of training data and move it to the device

          # optimizer.zero_grad()
          print('Batch #' + str(batch_idx + 1))
          for in_val in input:
            optimizer.zero_grad()
            # print(in_val[0])
            in_val = in_val.to(device).float()
            [decoded, given, mu, log_var] = model(in_val)
            # print(decoded[0])

            kld_loss = torch.mean(0.5 * torch.sum(1 + log_var - mu ** 2
                                                - log_var.exp()))
            recon_loss = criterion(decoded, in_val)
            loss = recon_loss + 0.0001 * kld_loss

            recon_losses.append(recon_loss.item())
            kld_losses.append(kld_loss.item())
            losses.append(loss.item())
            loss.backward()
            optimizer.step()


      # Print the epoch loss
      print("Epoch {}: Reconstruction Loss: {:.4f}, KLD Loss: {:.4f}".format(
          epoch + 1,
          np.mean(recon_losses),
          np.mean(kld_losses)
      ))
  model.eval()
  # Return the trained model
  return model



In [331]:
result_model = train_model(train_loader)
print('L2 Norm Distances Between Input and Decoded vectors')
for test_idx, test_input in enumerate(test_loader):
    test_input = test_input.to(device).float()
    [decoded, given, mu, log_var] = result_model(test_input)
    print(torch.sum(torch.abs(decoded - given)))
    # print('Decoded')
    # print(decoded)
    # print('Given')
    # print(given)

Batch #1
Batch #2
Batch #3
Batch #4
Batch #5
Batch #6
Batch #7
Batch #8
Batch #9
Batch #10
Batch #11
Batch #12
Batch #13
Batch #14
Batch #15
Batch #16
Batch #17
Batch #18
Batch #19
Batch #20
Batch #21
Batch #22
Batch #23
Batch #24
Batch #25
Batch #26
Batch #27
Batch #28
Batch #29
Batch #30
Batch #31
Batch #32
Batch #33
Batch #34
Batch #35
Batch #36
Batch #37
Batch #38
Batch #39
Batch #40
Batch #41
Batch #42
Batch #43
Batch #44
Batch #45
Batch #46
Batch #47
Batch #48
Batch #49
Batch #50
Batch #51
Batch #52
Batch #53
Batch #54
Batch #55
Batch #56
Batch #57
Batch #58
Batch #59
Batch #60
Batch #61
Batch #62
Batch #63
Batch #64
Batch #65
Batch #66
Batch #67
Batch #68
Batch #69
Batch #70
Batch #71
Batch #72
Batch #73
Batch #74
Batch #75
Batch #76
Batch #77
Batch #78
Batch #79
Batch #80
Batch #81
Batch #82
Batch #83
Batch #84
Batch #85
Batch #86
Batch #87
Batch #88
Batch #89
Batch #90
Batch #91
Batch #92
Batch #93
Batch #94
Batch #95
Batch #96
Batch #97
Batch #98
Batch #99
Batch #100
Batch #1